# **TRAINING**

In [ ]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'
os.environ['GRPC_VERBOSITY'] = 'ERROR'
os.environ['GRPC_TRACE'] = ''
os.environ['GRPC_DEFAULT_SSL_ROOTS_FILE_PATH'] = '/etc/ssl/certs/ca-certificates.crt'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import Model as tfkModel

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)
print(tfk.__version__)

# Import other libraries
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import register_keras_serializable
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
from keras.callbacks import Callback
import IPython.display as display
# import graphviz
from PIL import Image
import matplotlib.gridspec as gridspec
import json

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

## **DISTRIBUTION DEFINITIONS**

In [ ]:
def auto_select_accelerator():
    """
    Reference:
        * https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu
        * https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training
    """
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy

In [ ]:
# Setting che correct strategy for TPU / batch sizes
strategy = auto_select_accelerator()
numGPU = len(tf.config.list_physical_devices('GPU'))
numTPU = len(tf.config.list_logical_devices('TPU'))
print("Num GPUs Available: ", numGPU)
print("Num TPUs Available: ", numTPU)

In [ ]:
batch_size = 32
if numTPU != 0:
    batch_size = strategy.num_replicas_in_sync * 8

print(f"Batch size: {batch_size}")

## **DATA PREPROCESSING**

In [ ]:
data_path = "/kaggle/input/data-preprocessing/data_preprocessed.npz"

In [ ]:
data = np.load(data_path, allow_pickle=True)
lst = data.files
X = data[lst[0]]
y = data[lst[1]]

# Convert values in data to int
X = X.astype(int)

# Normalize data to the range [0, 1]
X = (X / 255).astype('float32')

# Create a mapping from label string to values
map = {'Basophil':0 , 'Eosinophil':1, 'Erythroblast': 2, 'Immature granulocytes': 3, 'Lymphocyte': 4, 'Monosyte': 5, 'Neutrophil': 6, 'Platelet': 7}

# Convert labels to categorical format using one-hot encoding
#y = tf.keras.utils.to_categorical(y)

# Encode the labels via LabelEncoding from stratch
y = np.array(map[label] for label in y)

# Split data into training, validation, and test sets, maintaining class distribution
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=0.1, stratify=y)

# Splitting the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=0.15, stratify=y_train_val)

In [ ]:
del X, y # to free up resources

In [ ]:
# Print the shapes of the loaded datasets
print("Training Data Shape:", X_train.shape)
print("Training Label Shape:", y_train.shape)
print("Validation Data Shape:", X_val.shape)
print("Validation Label Shape:", y_val.shape)
print("Test Data Shape:", X_test.shape)
print("Test Label Shape:", y_test.shape)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).cache().shuffle(4096).batch(batch_size).repeat().prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).cache().shuffle(4096).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).cache().shuffle(4096).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Input shape for the model
input_shape = X_train.shape[1:]

# Output shape for the model
output_shape = y_train.shape[1]

steps_per_epoch = y_train.shape[0] // batch_size

print("Input Shape: ", input_shape)
print("Output Shape: ", output_shape)
print("Steps per epoch: ", steps_per_epoch)

## **CUSTOM CALLBACKS AND METRICS DEFINITIONS**

In [ ]:
# Custom implementation of ReduceLROnPlateau
class CustomReduceLROnPlateau(tf.keras.callbacks.Callback):
    def __init__(self, monitor='val_binary_accuracy', factor=0.33, patience=20, min_lr=1e-8, verbose=1):
        super(CustomReduceLROnPlateau, self).__init__()
        self.monitor = monitor
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.verbose = verbose
        self.wait = 0
        self.best = None
        self.new_lr = None

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        
        # Initialize best metric if it's the first epoch
        if self.best is None:
            self.best = current
            return

        # Check if the monitored metric has improved
        if current > self.best:
            self.best = current
            self.wait = 0
        else:
            self.wait += 1

            # If patience is exceeded, reduce the learning rate
            if self.wait >= self.patience:
                old_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
                if old_lr == self.min_lr:
                    return
                self.new_lr = max(old_lr * self.factor, self.min_lr)
                self.model.optimizer.learning_rate.assign(self.new_lr)
                
                if self.verbose > 0:
                    print(f"\nEpoch {epoch + 1}: reducing learning rate to {self.new_lr}.")
                
                self.wait = 0  # Reset patience counter

In [ ]:
class DisplayLearningRateCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Get the current learning rate from the optimizer and display it
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        print(f"Epoch {epoch+1} : Learning rate = {tf.keras.backend.get_value(lr)}")

In [ ]:
class WeightedMacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name="weighted_macro_f1_score", **kwargs):
        super(WeightedMacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.true_positives = self.add_weight("tp", shape=(num_classes,), initializer="zeros")
        self.false_positives = self.add_weight("fp", shape=(num_classes,), initializer="zeros")
        self.false_negatives = self.add_weight("fn", shape=(num_classes,), initializer="zeros")
        self.support = self.add_weight("support", shape=(num_classes,), initializer="zeros")  # To track class weights

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.argmax(y_pred, axis=1)

        for i in range(self.num_classes):
            y_true_i = tf.equal(y_true, i)
            y_pred_i = tf.equal(y_pred, i)

            # True positives, false positives, and false negatives for class `i`
            tp = tf.reduce_sum(tf.cast(y_true_i & y_pred_i, tf.float32))
            fp = tf.reduce_sum(tf.cast(~y_true_i & y_pred_i, tf.float32))
            fn = tf.reduce_sum(tf.cast(y_true_i & ~y_pred_i, tf.float32))
            support = tf.reduce_sum(tf.cast(y_true_i, tf.float32))  # Total true samples for class `i`

            self.true_positives[i].assign_add(tp)
            self.false_positives[i].assign_add(fp)
            self.false_negatives[i].assign_add(fn)
            self.support[i].assign_add(support)

    def result(self):
        # Calculate F1 for each class and weight it
        f1_scores = []
        for i in range(self.num_classes):
            tp = self.true_positives[i]
            fp = self.false_positives[i]
            fn = self.false_negatives[i]
            support = self.support[i]

            precision = tp / (tp + fp + tf.keras.backend.epsilon())
            recall = tp / (tp + fn + tf.keras.backend.epsilon())
            f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
            f1_scores.append(f1 * (support / tf.reduce_sum(self.support)))

        return tf.reduce_sum(f1_scores)

    def reset_states(self):
        for v in self.variables:
            v.assign(tf.zeros_like(v))

## **MODEL DEFINITION**

In [ ]:
@register_keras_serializable()
class CustomCastLayer(tfk.layers.Layer):
    def call(self, inputs):
        return tf.cast(inputs * 255, tf.uint8)

@register_keras_serializable()
class CustomAugmentLayer(tfk.layers.Layer):
    def __init__(self, max_rotation=30.0, **kwargs):
        super(CustomAugmentLayer, self).__init__(**kwargs)
        self.max_rotation = max_rotation / 360.00
        
    def call(self, inputs, training=False):
        if training:
            inputs = tf.image.random_flip_up_down(tf.image.random_flip_left_right(inputs))
            inputs = tf.image.random_brightness(inputs, max_delta=0.1)
            inputs = tf.image.random_contrast(inputs, lower=0.9, upper=1.1)
            
        return inputs

In [ ]:
def create_model(shape=input_shape, n_labels=output_shape, convnext_trainable=False, #standard definitions
                 n_dense_layers=1, initial_dense_neurons=1024, min_neurons=64, # architecture definitions
                 include_dropout=False, dropout_rate=0.3, l2_lambda=0, # against overfitting
                 learning_rate=6e-4):
    
    # Seed for reproducibility
    tf.random.set_seed(seed)

    # Metrics definition
    weighted_macro_f1 = WeightedMacroF1Score(num_classes=n_labels)
    METRICS = [tfk.metrics.SparseCategoricalAccuracy(), weighted_macro_f1]

    # The input layer
    inputs = tfkl.Input(shape=input_shape, name='Input')

    # The two augmentation layers
    x = CustomCastLayer()(inputs)
    x = CustomAugmentLayer()(x, training=True)

    # The convnext layer with include top=False to take the convolutional part only
    convnext = tfk.applications.ConvNeXtXLarge(
                input_shape=input_shape,
                weights='imagenet',
                include_top=False
            )

    # Here we freeze the convnext to perform Tranfer Learning
    convnext.trainable = convnext_trainable

    x = convnext(x)
    x = tfkl.GlobalAveragePooling2D()(x)

    # Hidden layers building
    neurons = initial_dense_neurons
    for k in range(n_dense_layers):
        x = tfkl.Dense(units=neurons, activation='silu', name=f'Dense_layer_{k}', kernel_regularizer=tfk.regularizers.L2(l2_lambda))(x)
        if include_dropout:
            x = tfkl.Dropout(dropout_rate, name=f'Dropout_layer_{k}')(x)
        neurons = max(neurons // 2, min_neurons)
    outputs = tfkl.Dense(output_shape, activation='softmax', name='output_layer')(x)

    # Final model building
    model = tfk.Model(inputs=inputs, outputs=outputs, name='TF-CNN')

    # Compile the model
    loss = tfk.losses.SparseCategoricalCrossentropy()
    optimizer = tfk.optimizers.AdamW(learning_rate, weight_decay=l2_lambda)
    
    model.compile(loss=loss, optimizer=optimizer, metrics=METRICS)

    # Return the model
    return model

## **TRANSFER LEARNING**

In [ ]:
# Best values found so far
n_dense_layers = 2
dropout_rate = 0.35
include_dropout = True
l2_lambda = 1e-2

epochs = 100

In [ ]:
# Build the model with specified input and output shapes
with strategy.scope():
    model = create_model(n_dense_layers=n_dense_layers, include_dropout=True, dropout_rate=dropout_rate, l2_lambda=l2_lambda)

# Display a summary of the model architecture
model.summary(expand_nested=False, show_trainable=True)

In [ ]:
# Define the patience value for early stopping
patience = 35

# Create an EarlyStopping callback
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=patience,
    restore_best_weights=True
)

lr_reducer = CustomReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=25, min_lr=1e-8)
#plot_callback = RealTimePlot()

# Store the callback in a list
callbacks = [lr_reducer, DisplayLearningRateCallback()]

In [ ]:
# Train the model with early stopping callback
history = model.fit(
    train_dataset,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    shuffle=True,
    callbacks=callbacks
).history

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(15, 5))
plt.plot(history['loss'], label='Training loss', alpha=.8)
plt.plot(history['val_loss'], label='Validation loss', alpha=.8)
plt.title('Loss')
plt.legend()
plt.grid(alpha=.3)

# Plot training and validation accuracy
plt.figure(figsize=(15, 5))
plt.plot(history['binary_accuracy'], label='Training accuracy', alpha=.8)
plt.plot(history['val_binary_accuracy'], label='Validation accuracy', alpha=.8)
plt.title('Accuracy')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
# Save the trained model to a file with the accuracy included in the filename
with strategy.scope():
    model_weights_filename = 'MODEL.weights.h5'
    model.save_weights(model_weights_filename)

In [ ]:
LABELS = ['Basophil', 'Eosinophil', 'Erythroblast', 'Immature granulocytes', 'Lymphocyte', 'Monosyte', 'Neutrophil', 'Platelet']

## **TRANSFER LEARNING EVALUATIONS**

In [ ]:
def evaluations(model, ds, y_ds, labels, name):
    # Predict class probabilities and get predicted classes
    predictions = model.predict(ds, verbose=0)
    predictions = np.argmax(predictions, axis=-1)
    
    # Extract ground truth classes
    ds_gt = np.argmax(y_ds, axis=-1)
    
    # Calculate and display training set accuracy
    ds_accuracy = accuracy_score(ds_gt, predictions)
    print(f'Accuracy score over the {name} set: {round(ds_accuracy, 4)}')
    
    # Calculate and display training set precision
    ds_precision = precision_score(train_gt, train_predictions, average='weighted')
    print(f'Precision score over the {name} set: {round(ds_precision, 4)}')
    
    # Calculate and display training set recall
    ds_recall = recall_score(ds_gt, ds_predictions, average='weighted')
    print(f'Recall score over the {name} set: {round(ds_recall, 4)}')
    
    # Calculate and display training set F1 score
    ds_f1 = f1_score(ds_gt, ds_predictions, average='weighted')
    print(f'F1 score over the {name} set: {round(ds_f1, 4)}')
    
    # Compute the confusion matrix
    cm = confusion_matrix(ds_gt, ds_predictions)
    
    # Create labels combining confusion matrix values
    labels = np.array([f"{num}" for num in cm.flatten()]).reshape(cm.shape)
    
    # Plot the confusion matrix with class labels
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=labels, fmt='', xticklabels=labels, yticklabels=labels, cmap='Blues')
    plt.xlabel('True labels')
    plt.ylabel('Predicted labels')
    plt.show()

In [ ]:
evaluations(model, ds=X_train, y_ds=y_train, name='training')

In [ ]:
evaluations(model, ds=X_val, y_ds=y_val, name='validation')

In [ ]:
evaluations(model, ds=X_test, y_ds=y_test, name='test')

In [ ]:
del model

## **FINE TUNING**

In [ ]:
# Best values found so far
n_dense_layers = 2
dropout_rate = 0.35
include_dropout = True
l2_lambda = 1e-2

epochs = 30

In [ ]:
# Build the model with specified input and output shapes
with strategy.scope():
    model = create_model(n_dense_layers=n_dense_layers, include_dropout=True, dropout_rate=dropout_rate, l2_lambda=l2_lambda, convnext_trainable=True)

model.trainable = True
for layer in model.get_layer('convnext_xlarge').layers:
    if isinstance(layer, tfkl.LayerNormalization) or isinstance(layer, tfkl.Normalization):
        layer.trainable = False
        print(f"Layer {layer.name}, trainable {layer.trainable}")
    if type(layer).__name__ == 'LayerScale':
        layer.trainable = False
        print(f"Layer {layer.name}, trainable {layer.trainable}")

In [ ]:
model.summary()

In [ ]:
model.load_weights("MODEL.weights.h5")

In [ ]:
# Define the patience value for early stopping
patience = 12

# Create an EarlyStopping callback
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    mode='max',
    patience=patience,
    restore_best_weights=True
)

lr_reducer = CustomReduceLROnPlateau(monitor='val_binary_accuracy', factor=0.5, patience=5, min_lr=1e-8)
plot_callback = RealTimePlot()

# Store the callback in a list
callbacks = [lr_reducer]

In [ ]:
# Train the model with early stopping callback
history = model.fit(
    train_dataset,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    callbacks=callbacks
).history

## **FINE TUNING EVALUATIONS**

In [ ]:
evaluations(model, ds=X_train, y_ds=y_train, name='training')

In [ ]:
evaluations(model, ds=X_val, y_ds=y_val, name='validation')

In [ ]:
evaluations(model, ds=X_test, y_ds=y_test, name='test')